# Google ColabでのYOLOトレーニング

このノートブックはGoogle ColabでUltralyticsを使用したYOLOv8モデルのトレーニングをデモンストレーションします。COCO8サンプルデータセットとメモリ問題を避けるための最適化パラメータを使用します。

**所要時間**: 約5-10分

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

## 1. 環境設定

まず、GPUが有効になっているか確認します。

**設定方法**: メニュー → ランタイム → ランタイムのタイプを変更 → GPU を選択

In [ ]:
# GPU確認
import torch
print(f"GPU利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")

## 2. ライブラリのインストール

In [ ]:
!pip install ultralytics

## 3. ライブラリのインポート

In [ ]:
from ultralytics import YOLO
import torch

# デバイス確認
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"使用デバイス: {device}")

## 4. データセットの準備

COCO8サンプルデータセット（COCOデータセットの小さなサブセット）を使用します。Ultralyticsが自動でダウンロードします。

In [ ]:
# データセット確認
print("COCO8データセットを使用します")
print("トレーニング: 4画像, 検証: 4画像")
print("クラス: 80クラス (COCOの全クラス)")

## 5. モデルの準備

In [ ]:
# モデルをロード
model = YOLO('yolov8n.pt')  # 事前学習済みモデルをロード（トレーニング推奨）

print(f"モデル: yolov8n.pt")
print(f"パラメータ数: {sum(p.numel() for p in model.parameters()):,}")

## 6. トレーニングの実行

In [ ]:
# モデルをトレーニング
results = model.train(
    data='coco8.yaml',
    epochs=1,
    imgsz=320,
    batch=8,
    device=device,
    workers=0,
    amp=False,
    mosaic=0.0,
    verbose=False,
)

print("トレーニング完了!")

## 7. トレーニング結果の確認

In [ ]:
# 結果の確認
import os
results_dir = 'runs/detect/train'
if os.path.exists(results_dir):
    print("トレーニング結果が保存されました")
    print("保存されたファイル:")
    for file in os.listdir(results_dir):
        print(f"  - {file}")
else:
    print("結果ディレクトリが見つかりません")

## 8. 推論のテスト

In [ ]:
# 推論テスト
model = YOLO('runs/detect/train/weights/best.pt')

# サンプル画像で推論
results = model.predict('https://ultralytics.com/images/bus.jpg')

print("推論完了")
print(f"検出されたオブジェクト数: {len(results[0].boxes)}")

## 次のステップ

1. **データ量を増やす**: `data='coco128.yaml'` に変更
2. **エポック数を増やす**: `epochs=10`
3. **画像サイズを上げる**: `imgsz=640`
4. **カスタムデータセットを使う**: 独自のデータセットをアップロード
5. **モデルを保存**: Hugging Face Hubにアップロード